# TEST of custom SAC agent on easy environment

In [1]:
import gymnasium as gym
import seaborn as sns
import matplotlib.pyplot as plt

import torch

import numpy as np
import pandas as pd

# from custom_agent.CTCE.sac_agent import Agent
from custom_agents.CTCE_algorithms.single_agent_SAC_continuous import SAC

env = gym.make("Pendulum-v1")

sac_agent = SAC(env, batch_size=256, buffer_max_size=10000)


/home/hkolstee/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
logs_list = []

for i in range(1):
    # make agent
    sac_agent = SAC(env, batch_size=100, buffer_max_size=100000)
    
    # training run
    sac_agent.train(nr_steps = 15000, warmup_steps = 1000, learn_delay = 100, learn_freq = 1, learn_weight = 1)

torch.Size([100]) torch.Size([1]) torch.Size([100])
tensor(-1.)
torch.Size([100]) torch.Size([1]) torch.Size([100])
tensor(-1.)
torch.Size([100]) torch.Size([1]) torch.Size([100])
tensor(-1.)
torch.Size([100]) torch.Size([1]) torch.Size([100])
tensor(-1.)
torch.Size([100]) torch.Size([1]) torch.Size([100])
tensor(-1.)
torch.Size([100]) torch.Size([1]) torch.Size([100])
tensor(-1.)
torch.Size([100]) torch.Size([1]) torch.Size([100])
tensor(-1.)
torch.Size([100]) torch.Size([1]) torch.Size([100])
tensor(-1.)
torch.Size([100]) torch.Size([1]) torch.Size([100])
tensor(-1.)
torch.Size([100]) torch.Size([1]) torch.Size([100])
tensor(-1.)
torch.Size([100]) torch.Size([1]) torch.Size([100])
tensor(-1.)
torch.Size([100]) torch.Size([1]) torch.Size([100])
tensor(-1.)
torch.Size([100]) torch.Size([1]) torch.Size([100])
tensor(-1.)
torch.Size([100]) torch.Size([1]) torch.Size([100])
tensor(-1.)
torch.Size([100]) torch.Size([1]) torch.Size([100])
tensor(-1.)
torch.Size([100]) torch.Size([1]) torch.

KeyboardInterrupt: 

In [4]:
from gymnasium.utils.save_video import save_video
env2 = gym.make("Pendulum-v1", g=9.81, render_mode = "rgb_array_list")

# test loop
obs, info = env2.reset()
done = 0
truncated = 0
for _ in range(100):
    action = sac_agent.get_action(obs, False, True)

    next_obs, reward, done, truncated, info = env2.step(action)

    obs = next_obs

save_video(env2.render(), "videos", name_prefix = "custom_SAC_pendulum", fps=env.metadata["render_fps"], step_starting_index=0)

Moviepy - Building video /home/hkolstee/masterThesis/masterThesis/videos/custom_SAC_pendulum-episode-0.mp4.
Moviepy - Writing video /home/hkolstee/masterThesis/masterThesis/videos/custom_SAC_pendulum-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /home/hkolstee/masterThesis/masterThesis/videos/custom_SAC_pendulum-episode-0.mp4


# Somewhat more difficult env (still not very difficult): Half Cheetah V4

In [ ]:
import gymnasium as gym
import seaborn as sns
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

from custom_agent.CTCE.sac_agent_AE import Agent

In [ ]:
env = gym.make("HalfCheetah-v4")
print(env.observation_space)
print(env.observation_space.shape)
print(env.action_space)
print(env.action_space.high)
print(env.action_space.shape)

In [ ]:
logs_list = []

for i in range(3):
    # make agent
    sac_agent = Agent(env, batch_size=100, buffer_max_size=1000000)
    
    # training run
    logs = sac_agent.train(200000, warmup_steps=10000, learn_delay = 5000, learn_freq = 1, learn_weight = 1)
    
    # save training logs for this run
    logs_list.append(logs)

In [ ]:
logs = pd.concat(logs_list)

In [ ]:
display(logs[0:10])

In [ ]:
learn_delay = 5000

plt.figure()
sns.set(style="darkgrid", palette="rainbow")
plot = sns.lineplot(data = logs, x = "step", y = "avg_reward")
plot.set(title="SAC for HalfCheetah-v4", xlabel = "step", ylabel = "Episode Mean Reward")
plt.show()

plt.figure()
sns.set(style="darkgrid", palette="rainbow")
plot = sns.lineplot(data = logs[logs["step"] > learn_delay], x = "step", y = "avg_actor_loss")
plot.set(title="Actor Loss", xlabel = "Step", ylabel = "Loss")
plt.show()

plt.figure()
sns.set(style="darkgrid", palette="rainbow")
plot = sns.lineplot(data = logs[logs["step"] > learn_delay], x = "step", y = "avg_critic_loss")
plot.set(title="Critic Loss", xlabel = "Step", ylabel = "Loss")
plt.show()

plt.figure()
sns.set(style="darkgrid", palette="rainbow")
plot = sns.lineplot(data = logs[logs["step"] > learn_delay], x = "step", y = "avg_policy_entr")
plot.set(title="Policy Entropy", xlabel = "Step", ylabel = "Entropy")
plt.show()

plt.figure()
sns.set(style="darkgrid", context="notebook", palette="rainbow")
sns.lineplot(data = logs[logs["step"] > learn_delay], x = "step", y = "avg_alpha_loss").set(title="Entropy Coefficient (alpha) Loss", xlabel = "Step", ylabel = "Loss")

plt.figure()
sns.set(style="darkgrid", context="notebook", palette="rainbow")
sns.lineplot(data = logs[logs["step"] > learn_delay], x = "step", y = "avg_alpha").set(title="Entropy Coefficient (alpha)", xlabel = "Step", ylabel = "Value")

In [ ]:
from gymnasium.utils.save_video import save_video

env2 = gym.make("HalfCheetah-v4", render_mode = "rgb_array_list")

# test loop
obs, info = env2.reset()
done = 0
truncated = 0
while not (done or truncated):
    action = sac_agent.get_action(obs, False, True)

    next_obs, reward, done, truncated, info = env2.step(action)

    obs = next_obs

save_video(env2.render(), "videos", name_prefix = "custom_SAC_halfcheetahv4", fps=env.metadata["render_fps"], step_starting_index=0)

# Ant env

In [ ]:
import gymnasium as gym
import seaborn as sns
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

from custom_agent.CTCE.sac_agent_AE import Agent

In [ ]:
env = gym.make("Ant-v4")
print(env.observation_space)
print(env.observation_space.shape)
print(env.action_space)
print(env.action_space.high)
print(env.action_space.shape)

In [ ]:
logs_list = []
learn_delay = 5000

for i in range(1):
    # make agent
    sac_agent = Agent(env, batch_size=256, buffer_max_size=1000000)
    
    # training run
    logs = sac_agent.train(1000000, warmup_steps=10000, learn_delay = learn_delay, learn_freq = 1, learn_weight = 1)
    0
    # save training logs for this run
    logs_list.append(logs)

In [ ]:
logs = pd.concat(logs_list)

In [ ]:
display(logs[0:10])

In [ ]:
learn_delay = 5000

plt.figure()
sns.set(style="darkgrid", palette="rainbow")
plot = sns.lineplot(data = logs, x = "step", y = "avg_reward")
plot.set(title="SAC for Ant-v4", xlabel = "step", ylabel = "Episode Mean Reward")
plt.show()

plt.figure()
sns.set(style="darkgrid", palette="rainbow")
plot = sns.lineplot(data = logs[logs["step"] > learn_delay], x = "step", y = "avg_actor_loss")
plot.set(title="Actor Loss", xlabel = "Step", ylabel = "Loss")
plt.show()

plt.figure()
sns.set(style="darkgrid", palette="rainbow")
plot = sns.lineplot(data = logs[logs["step"] > learn_delay], x = "step", y = "avg_critic_loss")
plot.set(title="Critic Loss", xlabel = "Step", ylabel = "Loss")
plt.show()

plt.figure()
sns.set(style="darkgrid", palette="rainbow")
plot = sns.lineplot(data = logs[logs["step"] > learn_delay], x = "step", y = "avg_policy_entr")
plot.set(title="Policy Entropy", xlabel = "Step", ylabel = "Entropy")
plt.show()

plt.figure()
sns.set(style="darkgrid", context="notebook", palette="rainbow")
sns.lineplot(data = logs[logs["step"] > learn_delay], x = "step", y = "avg_alpha_loss").set(title="Entropy Coefficient (alpha) Loss", xlabel = "Step", ylabel = "Loss")

plt.figure()
sns.set(style="darkgrid", context="notebook", palette="rainbow")
sns.lineplot(data = logs[logs["step"] > learn_delay], x = "step", y = "avg_alpha").set(title="Entropy Coefficient (alpha)", xlabel = "Step", ylabel = "Value")

In [ ]:
from gymnasium.utils.save_video import save_video

env2 = gym.make("Ant-v4", render_mode = "rgb_array_list")

# test loop
obs, info = env2.reset()
done = 0
truncated = 0
while not (done or truncated):
    action = sac_agent.get_action(obs, False, True)

    next_obs, reward, done, truncated, info = env2.step(action)

    obs = next_obs

save_video(env2.render(), "videos", name_prefix = "antV4", fps=env.metadata["render_fps"], step_starting_index=0)

# Finally: CityLearn

In [ ]:
from custom_agent.CTCE.sac_agent import Agent
from custom_agent.CTCE.citylearn_wrapper import CityLearnWrapper

from citylearn.wrappers import NormalizedSpaceWrapper, NormalizedObservationWrapper
from custom_reward.custom_reward import CustomReward

from citylearn.citylearn import CityLearnEnv

In [ ]:
class WrapperEnv:
    """
    Env to wrap provide Citylearn Env data without providing full env
    Preventing attribute access outside of the available functions
    """
    def __init__(self, env_data):
        self.observation_names = env_data['observation_names']
        self.action_names = env_data['action_names']
        self.observation_space = env_data['observation_space']
        self.action_space = env_data['action_space']
        self.time_steps = env_data['time_steps']
        self.seconds_per_time_step = env_data['seconds_per_time_step']
        self.random_seed = env_data['random_seed']
        self.buildings_metadata = env_data['buildings_metadata']
        self.episode_tracker = env_data['episode_tracker']
    
    def get_metadata(self):
        return {'buildings': self.buildings_metadata}
    
def makeEnv(schema_path, reward_function):
    # create environment
    env = CityLearnEnv(schema = schema_path, reward_function = reward_function, central_agent=True)

    env_data = dict(
        observation_names = env.observation_names,
        action_names = env.action_names,
        observation_space = env.observation_space,
        action_space = env.action_space,
        time_steps = env.time_steps,
        random_seed = None,
        episode_tracker = None,
        seconds_per_time_step = None,
        buildings_metadata = env.get_metadata()['buildings']
    )

    wrapper_env = WrapperEnv(env_data)
    return env, wrapper_env

In [ ]:
schema_path = "data/schema.json"

env, wrapper_env = makeEnv(schema_path, CustomReward)
print(env.action_space)
# env = NormalizedObservationWrapper(env)
env = NormalizedSpaceWrapper(env)
env = CityLearnWrapper(env)
print(env.reset())

print(isinstance(env.unwrapped.reward_function, CustomReward))
print(env.unwrapped.reward_function.comfort[0])

In [ ]:
logs_list = []
learn_delay = 100

for i in range(1):
    # make agent
    sac_agent = Agent(env, batch_size=256, buffer_max_size=100000)
    # sac_agent = SAC("MlpPolicy", env)
    
    # training run
    sac_agent.train(3950000, warmup_steps=100, learn_delay = 100, learn_freq = 1, learn_weight = 1)

In [ ]:
from gymnasium.utils.save_video import save_video

# test loop
obs, info = env2.reset()
done = 0
truncated = 0
while not (done or truncated):
    action = sac_agent.get_action(obs, False, True)

    next_obs, reward, done, truncated, info = env2.step(action)

    obs = next_obs

save_video(env2.render(), "videos", name_prefix = "custom_SAC_halfcheetahv4", fps=env.metadata["render_fps"], step_starting_index=0)